# Job Board Scraping Lab

In this lab you will first see a minimal but fully functional code snippet to scrape the LinkedIn Job Search webpage. You will then work on top of the example code and complete several chanllenges.

### Some Resources 

- [Requests library](http://docs.python-requests.org/en/master/#the-user-guide) documentation 
- [Beautiful Soup Doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Urllib](https://docs.python.org/3/library/urllib.html#module-urllib)
- [re lib](https://docs.python.org/3/library/re.html)
- [Scrapy](https://scrapy.org/)
- [List of HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)
- [HTML basics](http://www.simplehtmlguide.com/cheatsheet.php)
- [CSS basics](https://www.cssbasics.com/#page_start)

In [1]:
# Import the required libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests

"""
This function searches job posts from LinkedIn and converts the results into a dataframe.
"""
def scrape_linkedin_job_search(keywords):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    # Assemble the full url with parameters
    scrape_url = ''.join([BASE_URL, 'keywords=', keywords])

    # Create a request to get the data from the server 
    page = requests.get(scrape_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)

    # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
    # Then in each job card, extract the job title, company, and location data.
    titles = []
    companies = []
    locations = []
    for card in soup.select("div.result-card__contents"):
        title = card.findChild("h3", recursive=False)
        company = card.findChild("h4", recursive=False)
        location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
        titles.append(title.string)
        companies.append(company.string)
        locations.append(location.string)
    
    # Inject job titles, companies, and locations into the empty dataframe
    zipped = zip(titles, companies, locations)
    for z in list(zipped):
        data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)
    
    # Return dataframe
    return data

In [2]:
# Example to call the function

results = scrape_linkedin_job_search('data%20analysis')
results

,Title,Company,Location
0,Delegation Coordinator (prior health plan expe...,Blue Cross Blue Shield of Arizona,"Phoenix, AZ, US"
1,Data Analysis Manager,Capital One,"McLean, VA, US"
2,Information Specialist II - Data Analysis,ECRI Institute,"Plymouth Meeting, PA, US"
3,Data Analysis Manager,Alliance of Community Health Plans (ACHP),Washington D.C. Metro Area
4,Principal Data Analyst – Catering Operations A...,Delta Air Lines,"Atlanta, GA, US"
5,Data Analysis Manager,Jobs @ TheJobNetwork,"Richmond, VA, US"
6,Staff Business Data Analysis,Intuit,"San Diego, CA, US"
7,JSEG Senior Systems & Data Analysis Engineer,Jacobs,"Huntsville, AL, US"
8,Data Analysis & Evaluation,AmeriCorps,"Phillipsburg, NJ, US"
9,Sales Data and Reporting Analysis,Kemper,"Dallas, TX, US"


## Challenge 1

The first challenge for you is to update the `scrape_linkedin_job_search` function by adding a new parameter called `num_pages`. This will allow you to search more than 25 jobs with this function. Suggested steps:

1. Go to https://www.linkedin.com/jobs/search/?keywords=data%20analysis in your browser.
1. Scroll down the left panel and click the page 2 link. Look at how the URL changes and identify the page offset parameter.
1. Add `num_pages` as a new param to the `scrape_linkedin_job_search` function. Update the function code so that it uses a "for" loop to retrieve several pages of search results.
1. Test your new function by scraping 5 pages of the search results.

Hint: Prepare for the case where there are less than 5 pages of search results. Your function should be robust enough to **not** trigger errors. Simply skip making additional searches and return all results if the search already reaches the end.

In [3]:
# your code here
def scrape_linkedin_job_search_pages(keywords, num_pages):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    
    # Assemble the full url with parameters, &start=50
    
    for i in range(num_pages):
    
        if i==0:
            scrape_url = ''.join([BASE_URL, 'keywords=', keywords])
        else:
            scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&start=', '{}'.format((i)*25)])

        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')


        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)

        # Inject job titles, companies, and locations into the empty dataframe
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data

In [30]:
results = scrape_linkedin_job_search_pages('data%20analysis', 2)
results

,Title,Company,Location
0,Data Analysis Manager,Capital One,"McLean, VA, US"
1,Information Specialist II - Data Analysis,ECRI Institute,"Plymouth Meeting, PA, US"
2,"Senior Analyst, Data and Analysis",Digitas North America,"Atlanta, Georgia"
3,Data Analysis Manager,Alliance of Community Health Plans (ACHP),Washington D.C. Metro Area
4,Population Health Analyst,Tech Finders,"Phoenix, Arizona"
5,Principal Data Analyst – Catering Operations A...,Delta Air Lines,"Atlanta, GA, US"
6,Data Analysis Manager,Jobs @ TheJobNetwork,"Richmond, VA, US"
7,Staff Business Data Analysis,Intuit,"San Diego, CA, US"
8,JSEG Senior Systems & Data Analysis Engineer,Jacobs,"Huntsville, AL, US"
9,Data Analysis & Evaluation,AmeriCorps,"Phillipsburg, NJ, US"


## Challenge 2

Further improve your function so that it can search jobs in a specific country. Add the 3rd param to your function called `country`. The steps are identical to those in Challange 1.

In [15]:
# your code here

def scrape_linkedin_job_search_pages_country(keywords, num_pages, country):
    
    # Define the base url to be scraped.
    # All uppercase variable name signifies this is a constant and its value should never unchange
    BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
    
    # Create an empty dataframe with the columns consisting of the information you want to capture
    columns = ['Title', 'Company', 'Location']
    data = pd.DataFrame(columns=columns)
    
    
    # Assemble the full url with parameters, &start=50
    
    for i in range(num_pages):
    
        if i==0:
            scrape_url = ''.join([BASE_URL, 'keywords=', keywords])
        else:
            scrape_url = ''.join([BASE_URL, 'keywords=', keywords, '&start=', '{}'.format((i)*25)])

        # Create a request to get the data from the server 
        page = requests.get(scrape_url)
        soup = BeautifulSoup(page.text, 'html.parser')


        # Retrieve HTML code from the webpage. Parse the HTML into a list of "cards".
        # Then in each job card, extract the job title, company, and location data.
        titles = []
        companies = []
        locations = []
        for card in soup.select("div.result-card__contents"):
            title = card.findChild("h3", recursive=False)
            company = card.findChild("h4", recursive=False)
            location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
            titles.append(title.string)
            companies.append(company.string)
            locations.append(location.string)

        # Inject job titles, companies, and locations into the empty dataframe
        zipped = zip(titles, companies, locations)
        for z in list(zipped):
            data=data.append({'Title' : z[0] , 'Company' : z[1], 'Location': z[2]} , ignore_index=True)

    # Return dataframe
    return data[data.Location.str.contains(country)]

In [32]:
results = scrape_linkedin_job_search_pages_country('data%20analysis', 2, 'TX')
results

,Title,Company,Location
34,Sales Data and Reporting Analysis,Kemper,"Dallas, TX, US"
43,Reporting Contractor (Data Analysis),VIP,"Plano, TX, US"
48,Research Data Collection and Analysis,Penn State University,"University Park, TX, US"


In [38]:
BASE_URL = 'https://www.linkedin.com/jobs/search/?'
    
scrape_url = ''.join([BASE_URL, 'keywords=', 'data%20analysis'])
    
page = requests.get(scrape_url)
soup = BeautifulSoup(page.text, 'html.parser')

print (soup.select("div.result-card__contents")[0].text)
'''
for card in soup.select("div.result-card__contents"):
    title = card.findChild("h3", recursive=False)
    company = card.findChild("h4", recursive=False)
    location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)
'''
 

Data Analysis ManagerCapital OneMcLean, VA, USAn ideal candidate will be on the leading edge of Analytical technology with a passion for the newest and most innovative tools. Still ...3 weeks ago


'\nfor card in soup.select("div.result-card__contents"):\n    title = card.findChild("h3", recursive=False)\n    company = card.findChild("h4", recursive=False)\n    location = card.findChild("span", attrs={"class": "job-result-card__location"}, recursive=True)\n'

## Challenge 3

Add the 4th param called `num_days` to your function to allow it to search jobs posted in the past X days. Note that in the LinkedIn job search the searched timespan is specified with the following param:

```
f_TPR=r259200
```

The number part in the param value is the number of seconds. 259,200 seconds equal to 3 days. You need to convert `num_days` to number of seconds and supply that info to LinkedIn job search.

In [6]:
# your code here

## Bonus Challenge

Allow your function to also retrieve the "Seniority Level" of each job searched. Note that the Seniority Level info is not in the initial search results. You need to make a separate search request for each job card based on the `currentJobId` value which you can extract from the job card HTML.

After you obtain the Seniority Level info, update the function and add it to a new column of the returned dataframe.

In [7]:
# your code here